In [36]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_html("https://fbref.com/pt/comps/24/2023/cronograma/2023-Serie-A-Resultados-e-Calendarios")[0]
pd.set_option("display.max_columns", None)
print(df)

      Sem  Dia        Data Horário              Em casa   xG Resultado  xG.1  \
0     1.0  sáb  2023-04-15   16:00         América (MG)  1.1       0–3   3.1   
1     1.0  sáb  2023-04-15   16:00            Palmeiras  3.0       2–1   1.0   
2     1.0  sáb  2023-04-15   18:30  Red Bull Bragantino  2.0       2–1   0.7   
3     1.0  sáb  2023-04-15   18:30        Botafogo (RJ)  0.9       2–1   2.0   
4     1.0  sáb  2023-04-15   18:30       Ath Paranaense  0.8       2–0   1.0   
..    ...  ...         ...     ...                  ...  ...       ...   ...   
421  38.0  qua  2023-12-06   21:30           Fluminense  2.4       2–3   2.4   
422  38.0  qua  2023-12-06   21:30            São Paulo  0.5       1–0   0.5   
423  38.0  qua  2023-12-06   21:30             Cruzeiro  1.6       1–1   1.4   
424  38.0  qua  2023-12-06   21:30        Internacional  1.2       3–1   0.4   
425  38.0  qua  2023-12-06   21:30        Vasco da Gama  1.6       2–1   0.8   

               Visitante  Público      

In [38]:
resultados = df[['Sem', 'Em casa', 'Resultado', 'Visitante']]
resultados

,Sem,Em casa,Resultado,Visitante
0,1.0,América (MG),0–3,Fluminense
1,1.0,Palmeiras,2–1,Cuiabá
2,1.0,Red Bull Bragantino,2–1,Bahia
3,1.0,Botafogo (RJ),2–1,São Paulo
4,1.0,Ath Paranaense,2–0,Goiás
...,...,...,...,...
421,38.0,Fluminense,2–3,Grêmio
422,38.0,São Paulo,1–0,Flamengo
423,38.0,Cruzeiro,1–1,Palmeiras
424,38.0,Internacional,3–1,Botafogo (RJ)


In [39]:
placar = resultados.loc[:,"Resultado"].str.split("–", n = 1, expand = True)
print(placar)
resultados.loc[:,"CasaResul"] = placar.loc[:,0]
resultados.loc[:,"VisiResul"] = placar.loc[:,1]
resultados = resultados.drop(columns=["Resultado"])
resultados

     0  1
0    0  3
1    2  1
2    2  1
3    2  1
4    2  0
..  .. ..
421  2  3
422  1  0
423  1  1
424  3  1
425  2  1

[426 rows x 2 columns]


C:\Users\lccalmon\AppData\Local\Temp\ipykernel_34644\3424367926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados.loc[:,"CasaResul"] = placar.loc[:,0]
C:\Users\lccalmon\AppData\Local\Temp\ipykernel_34644\3424367926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados.loc[:,"VisiResul"] = placar.loc[:,1]


,Sem,Em casa,Visitante,CasaResul,VisiResul
0,1.0,América (MG),Fluminense,0,3
1,1.0,Palmeiras,Cuiabá,2,1
2,1.0,Red Bull Bragantino,Bahia,2,1
3,1.0,Botafogo (RJ),São Paulo,2,1
4,1.0,Ath Paranaense,Goiás,2,0
...,...,...,...,...,...
421,38.0,Fluminense,Grêmio,2,3
422,38.0,São Paulo,Flamengo,1,0
423,38.0,Cruzeiro,Palmeiras,1,1
424,38.0,Internacional,Botafogo (RJ),3,1


In [41]:
resultados["Vencedor"] = np.where(resultados["CasaResul"] > resultados["VisiResul"], "C", 
                                  np.where(resultados["CasaResul"] < resultados["VisiResul"], "V", "E"))
resultados

,Sem,Em casa,Visitante,CasaResul,VisiResul,Vencedor
0,1.0,América (MG),Fluminense,0,3,V
1,1.0,Palmeiras,Cuiabá,2,1,C
2,1.0,Red Bull Bragantino,Bahia,2,1,C
3,1.0,Botafogo (RJ),São Paulo,2,1,C
4,1.0,Ath Paranaense,Goiás,2,0,C
...,...,...,...,...,...,...
421,38.0,Fluminense,Grêmio,2,3,V
422,38.0,São Paulo,Flamengo,1,0,C
423,38.0,Cruzeiro,Palmeiras,1,1,E
424,38.0,Internacional,Botafogo (RJ),3,1,C
